In [1]:
#Test with real_video
import matplotlib.pyplot as plt
import os
import random
import time

import tensorflow as tf
import cv2
import numpy as np
import imutils

from segmentation_models.metrics import iou_score
from segmentation_models import Unet
import segmentation_models as sm


sm.set_framework("tf.keras")
sm.framework()

#Initialize parameters
BACKBONE = "resnet34"
preprocess_input = sm.get_preprocessing(BACKBONE)
batch_size = 16
w, h = 384, 384

#initialize model
opt=tf.keras.optimizers.Adam(0.0001)
model= Unet(BACKBONE,encoder_weights = "imagenet",classes = 3,activation="softmax",input_shape = (w, h,3),encoder_freeze = True)
loss1 = sm.losses.categorical_focal_dice_loss

model.compile(optimizer=opt,loss=loss1,metrics=[iou_score])

model.load_weights("best2.hdf5")


Segmentation Models: using `keras` framework.


In [2]:
'''
from keras.models import load_model 
model = load_model("best2.h5", compile = False)
'''

In [5]:
import cv2
import time
import numpy as np

#id = 'http://192.168.1.2:4747/video'
id = 'Polyp_1.mp4'
w, h = 384, 384
cap = cv2.VideoCapture(id)
cap1 = cv2.VideoCapture('culture.mp4')
rotate = 0
#writer= cv2.VideoWriter('Polyp_1_detected.avi', cv2.VideoWriter_fourcc(*'DIVX'), 20, (w,h))

while True:
    start = time.time()
    success, frame = cap.read()
    if success:
        frame = cv2.resize(frame, (w, h))
        # Predict mask with trained model

        mask_predict = model.predict(frame[np.newaxis, :, :, :])
        mask = mask_predict[0]#[:, :, 0]
        print(mask.shape)

        img2 = np.zeros_like(frame)
        img2[:, :, 0] = mask[:, :, 0] * 0
        img2[:, :, 1] = mask[:, :, 1] * 255
        img2[:, :, 2] = mask[:, :, 2] * 255

        #Count FPS
        fps = 1 / (time.time() - start)
        print(start)

        #Draw mask
        result = cv2.bitwise_or(frame, img2)

        #Show prediction
        #im = cv2.hconcat([frame, img2, Result])
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(result, "FPS: " + str(int(fps)), (5, 20), font, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.imshow("Result", result)
        #writer.write(result)
        
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
#writer.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619818.582095
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619818.6672082
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619818.7285259
1/1 [==============================] - 0s 16ms/step
(384, 384, 3)
1678619818.790635
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619818.8534653
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619818.9151938
1/1 [==============================] - 0s 16ms/step
(384, 384, 3)
1678619818.9777741
1/1 [==============================] - 0s 16ms/step
(384, 384, 3)
1678619819.040577
1/1 [==============================] - 0s 16ms/step
(384, 384, 3)
1678619819.101514
1/1 [==============================] - 0s 19ms/step
(384, 384, 3)
1678619819.1636386
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
1678619819.2268517
1/1 [==============================] - 0s 17ms/step
(384, 384, 3)
167